# Load Libraries

In [1]:
import os
current = os.getcwd()
os.chdir("../")

In [2]:
import pandas as pd
import talib
import numpy as np #computing multidimensionla arrays
import datetime
import urllib3
import time
import settings

In [3]:
os.chdir(current)

## logging

In [8]:
import logging

# logging.basicConfig(filename='null',format='%(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#create a file handler
handler = logging.FileHandler('/tmp/logfile.log')
handler.setLevel(logging.INFO)


In [9]:
title = logging.Formatter('%(message)s')
handler.setFormatter(title)

# add the handlers to the logger
logger.addHandler(handler)


## Load binance libraries

In [11]:
from binance.client import Client
api_key = settings.api_key
api_secret = settings.api_secret
client = Client(api_key, api_secret)

# StochasticRSI Function
def Stoch(close,high,low, smoothk, smoothd, n):
    lowestlow = pd.Series.rolling(low,window=n,center=False).min()
    highesthigh = pd.Series.rolling(high, window=n, center=False).max()
    K = pd.Series.rolling(100*((close-lowestlow)/(highesthigh-lowestlow)), window=smoothk).mean()
    D = pd.Series.rolling(K, window=smoothd).mean()
    return K, D


## Get the historical data

The get_historical_klines function will take a while probably it's better to paginate it ToDo

In [12]:
start_date = "1 Apr, 2021"

In [39]:
#     # Get Binance Data into dataframe
#     # candles = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1MINUTE)
candles = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, start_date)

The operations below set the names for the columns in the dataframe

In [40]:
df = pd.DataFrame(candles)
df.columns=['timestart','open','high','low','close','?','timeend','?','?','?','?','?']

In [35]:
final_df = pd.DataFrame(columns = ["candlestart", "candleend", "candleclose", "candleRSI", "candleK", "candleD"])

In [ ]:
for i in range(0, len(df)-500 + 1):
    start = i
    end = i + 500 - 1
    current_df = df[start:end]
    
    # The operation below translates the timestamps into datetime format
    current_df.timestart = [datetime.datetime.fromtimestamp(i/1000) for i in current_df.timestart.values]
    current_df.timeend = [datetime.datetime.fromtimestamp(i/1000) for i in current_df.timeend.values]
    
    # This seems to be computing the RSI
    float_data = [float(x) for x in current_df.close.values]
    np_float_data = np.array(float_data)
    rsi = talib.RSI(np_float_data, 14)
    current_df['rsi'] = rsi

    # Compute StochRSI using RSI values in Stochastic function
    mystochrsi = Stoch(current_df.rsi, current_df.rsi, current_df.rsi, 3, 3, 14)
    current_df['MyStochrsiK'], current_df['MyStochrsiD'] = mystochrsi

    newestcandlestart = current_df.timestart.astype(str).iloc[-1] #gets last time
    newestcandleend = current_df.timeend.astype(str).iloc[-1] #gets current time?
    newestcandleclose = current_df.close.iloc[-1] #gets last close
    newestcandleRSI = current_df.rsi.astype(str).iloc[-1] #gets last rsi
    newestcandleK = current_df.MyStochrsiK.astype(str).iloc[-1] #gets last rsi
    newestcandleD = current_df.MyStochrsiD.astype(str).iloc[-1] #gets last rsi
    
    final_df.append({"candlestart": newestcandlestart, 
                     "candleend": newestcandleend, 
                     "candleclose": newestcandleclose, 
                     "candleRSI": newestcandleRSI, 
                     "candleK": newestcandleK, 
                     "candleD": newestcandleD}, ignore_index=True)

<ipython-input-43-2b6fe7f96fca>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df['rsi'] = rsi
<ipython-input-43-2b6fe7f96fca>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df['MyStochrsiK'], current_df['MyStochrsiD'] = mystochrsi
